In [1]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
import math
import torch
from copy import deepcopy
from itertools import chain 
from torch.utils.data import Dataset
from torchtext.vocab import vocab as Vocab

import warnings
warnings.filterwarnings("ignore")


Pathing


In [2]:
#Lokalt
data_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\data'
ab_dir = 'c:\\Users\\erika\\Desktop\\Exjobb\\repo\\base'

In [2]:
#stationär
data_dir = 'c:\\Users\\erikw\\Desktop\\Exjobb kod\\data'
ab_dir = 'c:\\Users\\erikw\\Desktop\\Exjobb kod\\base'

In [ ]:
#saga
data_dir = "/home/aeerik/data/raw/"


CONFIG

In [28]:
# Budget config file
include_pheno = False
threshold_year = 1970
data_path = data_dir
ab_path = ab_dir
max_length = [51,37]
mask_prob = 0.15
embedding_dim = 32
drop_prob = 0.2

#Encoder
dim_emb = 128
dim_hidden = 128
attention_heads = 8 

#BERT
num_encoders = 2

#trainer
epochs = 5
batch_size = 32
lr = 0.001
stop_patience = 10


Vocabulary

In [100]:
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
include_pheno = False
vocabulary = vocab_geno(NCBI, include_pheno)
vocab = vocab_pheno(ab_df)
print(len(vocabulary))
print(len(vocab))


1208
81


In [4]:
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
from data_preprocessing import data_loader
from create_dataset import NCBIDataset

include_pheno = True
threshold_year = 1970

data_path = data_dir
ab_path = ab_dir

NCBI,ab_df = data_loader(include_pheno,threshold_year,data_path,ab_path)

max_length = [51,37]
mask_prob = 0.25
vocabulary_geno = vocab_geno(NCBI, include_pheno)
vocabulary_pheno = vocab_pheno(ab_df)

test_set = NCBIDataset(NCBI, vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
test_set.prepare_dataset()

In [7]:
test_tensor = test_set[5][3]
print(test_tensor)
test_tensor = test_tensor[test_tensor != -1]
print(  test_tensor.tolist()  )

selected = res_pred[test_tensor.tolist() ]
print(selected)
expected = test_set[5][4]
expected = expected[expected != -1] 
print(expected)

tensor([78, 45, 72, 16, 69, 25, 30,  2, 42, 29, 22, 66,  3, 54,  5, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1])
[78, 45, 72, 16, 69, 25, 30, 2, 42, 29, 22, 66, 3, 54, 5]


IndexError: index 78 is out of bounds for dimension 0 with size 16

In [25]:
from torch.utils.data import DataLoader 
from bert_builder import BERT_ft

loss_function = torch.nn.BCEWithLogitsLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loader = DataLoader(test_set, batch_size=16, shuffle=False)
for i, batch in enumerate(loader):
    if i >= 1:
        break 
    input, token_target, attn_mask, AB_idx, SR_class  = batch
    bert_test = BERT_ft(vocab_size=len(vocabulary_geno), max_length=51, dim_embedding = 128, dim_hidden= 128, attention_heads=8, num_encoders=2, dropout_prob=0.2, num_ab=81, device=device)
    token_pred, res_pred = bert_test.forward(input, attn_mask)
    res_pred = torch.where(res_pred > 0, torch.ones_like(res_pred), torch.zeros_like(res_pred))
    list_AB_predictions = []
    for i, row in enumerate(res_pred):
        AB_list = 0
        AB_list = [elem.item() for elem in AB_idx[i] if elem.item() != -1]
        current_abs = []
        for ab in AB_list:
            current_abs.append(row[ab].item())
        current_abs = torch.tensor(current_abs)
        current_abs = current_abs.type(torch.int16)
        list_AB_predictions.append(current_abs)
    
        processed_tensor = [row[row != -1] for row in SR_class]
    for i, row in enumerate(processed_tensor):
        print(row)
        print(list_AB_predictions[i])
        correct = (row == list_AB_predictions[i]).sum().item()
        print(correct)
        print(len(row))

tensor([1])
tensor([1], dtype=torch.int16)
1
1
tensor([1])
tensor([0], dtype=torch.int16)
0
1
tensor([0])
tensor([1], dtype=torch.int16)
0
1
tensor([0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1])
tensor([1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
       dtype=torch.int16)
10
24
tensor([0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
        1])
tensor([1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
        0], dtype=torch.int16)
12
25
tensor([0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0])
tensor([0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1], dtype=torch.int16)
9
15
tensor([0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0])
tensor([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1], dtype=torch.int16)
8
15
tensor([0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0])
tensor([0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1], dtype=torch.int16)
8
15
tensor([0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0])
tensor

In [1]:
test = res_pred[14]
print(test)
pred_res = torch.where(test > 0, torch.ones_like(test), torch.zeros_like(test))
print(pred_res)
indicies = [1,2,3,4]
test = pred_res[indicies]
print)

NameError: name 'res_pred' is not defined

Trainer

In [ ]:
import numpy as np
from create_dataset import NCBIDataset
def get_split_indices(size_to_split, val_share, random_state: int = 42):
    indices = np.arange(size_to_split)
    np.random.seed(random_state)
    np.random.shuffle(indices)
    
    train_share = 1 - val_share
    
    train_size = int(train_share * size_to_split)
    
    train_indices = indices[:train_size]
    val_indices = indices[train_size:]
    
    return train_indices, val_indices
max_length = 20
mask_prob = 0.30
vocabulary = make_vocabulary(NCBI)

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary, max_length, mask_prob)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary, max_length, mask_prob)

In [33]:
import time
from torch import nn
from torch.utils.data import DataLoader
import wandb
from pathlib import Path

class BertTrainer_ft:
    def __init__(self, model, train_set, val_set, epochs, batch_size, lr, device, stop_patience, wandb_mode, project_name, wandb_name):
        
        self.model = model
        self.train_set = train_set
        self.train_size = len(train_set)
        self.val_set = val_set
        self.epochs = epochs    
        self.batch_size = batch_size
        self.num_batches = self.train_size // self.batch_size
        self.lr = lr
        self.weight_decay = 0.01
        self.current_epoch  = 0
        self.early_stopping_counter = 0	
        self.patience = stop_patience
        
        self.wandb_mode = wandb_mode
        self.project_name = project_name
        self.wandb_name = wandb_name
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        self.token_criterion = nn.CrossEntropyLoss(ignore_index = -1).to(device)
        self.ab_criterion = nn.BCEWithLogitsLoss().to(device)

        self.device = device


    def __call__(self):   
        if self.wandb_mode:
            self._init_wandb()   
        self.val_set.prepare_dataset() 
        self.val_loader = DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False)
        start_time = time.time()
        self.best_val_loss = float('inf')
        self._init_result_lists()
        for self.current_epoch in range(self.current_epoch, self.epochs):
            #Training
            self.model.train()
            self.train_set.prepare_dataset()
            self.train_loader = DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True)
            epoch_start_time = time.time()
            avg_epoch_loss_total, avg_epoch_loss_geno, avg_epoch_loss_ab = self.train(self.current_epoch)
            self.train_losses_total.append(avg_epoch_loss_total)
            self.train_losses_geno.append(avg_epoch_loss_geno) 
            self.train_losses_ab.append(avg_epoch_loss_ab)  
            print(f"Epoch completed in {(time.time() - epoch_start_time)/60:.1f} min")
            
            #Validation
            print("Evaluating on validation set...")
            val_results = self.evaluate(self.val_loader)
            print(f"Elapsed time: {time.strftime('%H:%M:%S', time.gmtime(time.time() - start_time))}")
            self.val_losses_total.append(val_results[0])  
            self.val_losses_geno.append(val_results[1])
            self.val_losses_ab.append(val_results[2])
            self.val_accs.append(val_results[3])
            if self.wandb_mode:
                self._report_epoch_results()
            criterion = self.stop_early()
            if criterion:
                print(f"Training interrupted at epoch: {self.current_epoch+1}")
                break
        print(f"-=Training completed=-")
        results = {
            "best_epoch": self.best_epoch,
            "total_train_losses": self.train_lossestotal,
            "geno_train_losses": self.train_losses_geno,
            "ab_train_losses": self.train_losses_ab,
            "total_val_losses": self.val_losses_total,
            "geno_val_losses": self.val_losses_geno,
            "ab_val_losses": self.val_losses_ab,
            "val_accs": self.val_accs
        }
        return results

    def _init_result_lists(self):

        self.train_losses_total = []
        self.train_losses_geno = []
        self.train_losses_ab = []

        self.val_losses_total = []
        self.val_losses_geno = []
        self.val_losses_ab = []

        self.val_accs = []
    
    def stop_early(self):
        if self.val_losses[-1] < self.best_val_loss:
            self.best_val_loss = self.val_losses[-1]
            self.best_epoch = self.current_epoch
            self.best_model_state = self.model.state_dict()
            self.early_stopping_counter = 0
            return False
        else:
            self.early_stopping_counter += 1
            return True if self.early_stopping_counter >= self.patience else False

    def train(self, epoch: int):
        print(f"Epoch {epoch+1}/{self.epochs}")
        time_ref = time.time()
        
        epoch_loss_total = 0
        epoch_loss_geno = 0
        epoch_loss_ab = 0

        for i, batch in enumerate(self.train_loader):
            input, token_target, attn_mask, AB_idx, SR_class = batch

            ABinclusion = torch.unique(AB_idx)
            ABinclusion = ABinclusion[ABinclusion != -1]
            ABinclusion = ABinclusion.tolist()
            self.model.exclude_networks(ABinclusion)

            self.optimizer.zero_grad() 

            token_predictions, resistance_predictions = self.model(input, attn_mask) 
            geno_loss = self.token_criterion(token_predictions.transpose(-1, -2), token_target) 
            
            list_AB_predictions = []
            for i, row in enumerate(resistance_predictions):
                AB_list = 0
                AB_list = [elem.item() for elem in AB_idx[i] if elem.item() != -1]
                current_abs = []
                for ab in AB_list:
                    current_abs.append(row[ab].item())
                current_abs = torch.tensor(current_abs)
                list_AB_predictions.append(current_abs)
            
                processed_tensor = [row[row != -1] for row in SR_class]
                for i, row in enumerate(processed_tensor):
                    row = row.type(torch.float32)
                    ab_loss =+ self.ab_criterion(list_AB_predictions[i], row)
            
            avg_total_loss = geno_loss.item() + ab_loss.item() /2
            epoch_loss_total += avg_total_loss 
            epoch_loss_geno += geno_loss.item()
            epoch_loss_ab += ab_loss.item()
        
            avg_total_loss.backward() 
            self.optimizer.step()
            self.model.reset_exclusion()         
        avg_epoch_loss_total = epoch_loss_total / self.num_batches
        avg_epoch_loss_geno = epoch_loss_geno / self.num_batches
        avg_epoch_loss_ab = epoch_loss_ab / self.num_batches

        return avg_epoch_loss_total, avg_epoch_loss_geno, avg_epoch_loss_ab
    
    def evaluate(self, loader):
        self.model.eval()
        epoch_loss_total = 0
        epoch_loss_geno =0
        epoch_loss_ab = 0
        total_correct = 0
        total_sum = 0

        with torch.no_grad():
            for i, batch in enumerate(loader):
                input, token_target, attn_mask, AB_idx, SR_class = batch

                token_predictions, resistance_predictions = self.model(input, attn_mask) 
                geno_loss = self.token_criterion(token_predictions.transpose(-1, -2), token_target) 
                
                list_AB_predictions = []
                for i, row in enumerate(resistance_predictions):
                    AB_list = 0
                    AB_list = [elem.item() for elem in AB_idx[i] if elem.item() != -1]
                    current_abs = []
                    for ab in AB_list:
                        current_abs.append(row[ab].item())
                    current_abs = torch.tensor(current_abs)
                    list_AB_predictions.append(current_abs)
                
                    processed_tensor = [row[row != -1] for row in SR_class]
                    for i, row in enumerate(processed_tensor):
                        row = row.type(torch.float32)
                        ab_loss =+ self.ab_criterion(list_AB_predictions[i], row)
                
                avg_total_loss = geno_loss.item() + ab_loss.item() /2
                epoch_loss_total += avg_total_loss 
                epoch_loss_geno += geno_loss.item()
                epoch_loss_ab += ab_loss.item() 
                
                list_AB_predictions = []
                for i, row in enumerate(resistance_predictions):
                    AB_list = 0
                    AB_list = [elem.item() for elem in AB_idx[i] if elem.item() != -1]
                    current_abs = []
                    for ab in AB_list:
                        current_abs.append(row[ab].item())
                    current_abs = torch.tensor(current_abs)
                    current_abs = current_abs.type(torch.int16)
                    list_AB_predictions.append(current_abs)
                
                    processed_tensor = [row[row != -1] for row in SR_class]
                for i, row in enumerate(processed_tensor):
                    total_correct += (row == list_AB_predictions[i]).sum().item()
                    total_sum += len(row)

        avg_epoch_loss_total = epoch_loss_total / self.num_batches
        avg_epoch_loss_geno = epoch_loss_geno / self.num_batches
        avg_epoch_loss_ab = epoch_loss_ab / self.num_batches

        accuracy = total_correct / total_sum

        return avg_epoch_loss_total, avg_epoch_loss_geno, avg_epoch_loss_ab, accuracy
    
    def _save_model(self, savepath: Path):
        torch.save(self.best_model_state, savepath)
        print(f"Model saved to {savepath}")
        
        
    def _load_model(self, savepath: Path):
        print(f"Loading model from {savepath}")
        self.model.load_state_dict(torch.load(savepath))
        print("Model loaded")

    def _init_wandb(self):
        self.wandb_run = wandb.init(
            project=self.project_name, # name of the project
            name=self.wandb_name, # name of the run
            
            config={
                "epochs": self.epochs,
                "batch_size": self.batch_size,
                "num_heads": self.model.attention_heads,
                "num_encoders": self.model.num_encoders,
                "emb_dim": self.model.dim_embedding,
                'ff_dim': self.model.dim_embedding,
                "lr": self.lr,
                "weight_decay": self.weight_decay,
                "max_seq_len": self.model.max_length[0],
                "vocab_size": len(self.train_set.vocab_geno),
                "num_parameters": sum(p.numel() for p in self.model.parameters() if p.requires_grad),
            }
        )
        self.wandb_run.watch(self.model) # watch the model for gradients and parameters
        self.wandb_run.define_metric("epoch", hidden=True)
        self.wandb_run.define_metric("batch", hidden=True)

        self.wandb_run.define_metric("TotalLosses/total_train_loss", summary="min", step_metric="epoch")
        self.wandb_run.define_metric("TotalLosses/total_val_loss", summary="min", step_metric="epoch")

        self.wandb_run.define_metric("GenoLosses/geno_train_loss", summary="min", step_metric="epoch")
        self.wandb_run.define_metric("GenoLosses/geno_val_loss", summary="min", step_metric="epoch")

        self.wandb_run.define_metric("AB_Losses/ab_train_loss", summary="min", step_metric="epoch")
        self.wandb_run.define_metric("AB_Losses/ab_val_loss", summary="min", step_metric="epoch")

        self.wandb_run.define_metric("Accuracies/val_acc", summary="min", step_metric="epoch")
        
        self.wandb_run.define_metric("Losses/final_val_loss")
        self.wandb_run.define_metric("Accuracies/final_val_acc")
        self.wandb_run.define_metric("final_epoch")

        return self.wandb_run
    
    def _report_epoch_results(self):
        wandb_dict = {
            "epoch": self.current_epoch+1,
            "TotalLosses/total_train_loss": self.train_losses_total[-1],
            "GenoLosses/geno_train_loss": self.train_losses_geno[-1],
            "ABLosses/ab_train_loss": self.train_losses_ab[-1],

            "TotalLosses/total_val_loss": self.val_losses_total[-1],
            "GenoLosses/geno_val_loss": self.val_losses_geno[-1],
            "ABLosses/ab_val_loss": self.val_losses_ab[-1],
            
            "Accuracies/val_acc": self.val_accs[-1],
        }
        self.wandb_run.log(wandb_dict)

In [ ]:
NCBI,ab_df = data_loader(include_pheno,threshold_year,data_path,ab_path)

max_length = [51,37]
mask_prob = 0.25
vocabulary_geno = vocab_geno(NCBI, include_pheno)
vocabulary_pheno = vocab_pheno(ab_df)

test_set = NCBIDataset(NCBI, vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
test_set.prepare_dataset()

In [35]:
from bert_builder import BERT_ft
from data_preprocessing import data_loader
from build_vocabulary import vocab_geno
from build_vocabulary import vocab_pheno
from misc import get_split_indices
from create_dataset import NCBIDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

include_pheno = True

NCBI,ab_df = data_loader(include_pheno,threshold_year,data_path,ab_path)
vocabulary_geno = vocab_geno(NCBI, include_pheno)
vocabulary_pheno = vocab_pheno(ab_df)

reduced_samples = 1000
NCBI = NCBI.head(reduced_samples)

print(f"Data loaded, {len(NCBI)} samples found")
print(f"length  of token vocabulary:",len(vocabulary_geno))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_indices, val_indices = get_split_indices(len(NCBI), 0.2)
train_set = NCBIDataset(NCBI.iloc[train_indices], vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)
val_set = NCBIDataset(NCBI.iloc[val_indices], vocabulary_geno, vocabulary_pheno, max_length, mask_prob,include_pheno)

bert_test = BERT_ft(len(vocabulary_geno), max_length, dim_emb, dim_hidden, attention_heads, num_encoders, drop_prob, len(vocabulary_pheno), device)

test = BertTrainer_ft(bert_test, train_set, val_set, epochs, batch_size, lr, device, stop_patience, False, "NCBI", "test")
test()

Data loaded, 1000 samples found
length  of token vocabulary: 479
Epoch 1/5


IndexError: list index out of range

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

------------------------------

In [ ]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()